# Face Generation 🎭

![](https://images.unsplash.com/photo-1528642474498-1af0c17fd8c3?ixlib=rb-1.2.1&ixid=eyJhcHBfaWQiOjEyMDd9&auto=format&fit=crop&w=1050&q=80)

Photo by [Ryoji Iwata](https://unsplash.com/photos/IBaVuZsJJTo)

---

# A GAN to generate faces

For what comes next, it might be good to have a clean code under the form of a class. This part is given to you. Make sure to understand it, since you'll be asked to use it afterwards.

In [18]:
class GAN():
    def __init__(self, x, y, z):

        self.img_rows = x
        self.img_cols = y
        self.channels = z
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        
        optimizer = Adam(0.0002, 0.5)
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(
            loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        self.generator = self.build_generator()
        self.generator.compile(
            loss='binary_crossentropy', 
            optimizer=optimizer)

        z = Input(shape=(100,))
        img = self.generator(z)

        self.discriminator.trainable = False
        valid = self.discriminator(img)

        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        noise_shape = (100,)

        model = Sequential()
        model.add(Dense(256, input_shape=noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()
        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, X_train, epochs, batch_size=128, save_interval=50):

        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        #X_train = np.expand_dims(X_train, axis=3)
        half_batch = int(batch_size / 2)

        for epoch in range(epochs):

            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (half_batch, 100))
            gen_imgs = self.generator.predict(noise)

            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            noise = np.random.normal(0, 1, (batch_size, 100))
            valid_y = np.array([1] * batch_size)

            g_loss = self.combined.train_on_batch(noise, valid_y)

            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5,5
        noise = np.random.normal(0, 1, (r * c, 100))
        gen_imgs = self.generator.predict(noise)

        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images_gan_face/face_%d.png" % epoch)
        plt.close()

GANs typically require between 10'000 and 100'000 iterations depending on the complexity of the underlying data. For this reason, running such models locally on CPUs is not such a good idea. 

We'll be using Google Colab's GPUs for this task. 

In this exercise, we'll take as inputs faces from actors of IMdB. The dataset has been already extracted and is currently under the form of a numpy array. The main steps of the exercise are to :
- import the data
- train a GAN on it
- generate new faces

### Step 1 : Google Colab

Open https://colab.research.google.com

### Step 2 : New notebook

Create a new Notebook in Python 3 as under :

<img src='../images/colab.png'>

### Step 3 : Download the data

Now, download the numpy array which contains the faces, and move the file to your Drive (Typically Drive/My Drive)

### Step 4 : Start the notebook

In your notebook, go to `Execution > Modifier le type d'exécution` and set it to GPU :

<img src='../images/gpu.png'>

### Step 5 : Allow colab to access your Drive

- In the notebook, type the following command to allow Colab to access your drive's files :

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

- Once you run the cell, a link is provided, similar to : Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=XXX

- Open the URL. You should now see a page asking for an access to the content of your Drive. Allow access.

- Now, simply copy the code given, and paste it in the cell of your notebook :

### Step 6 : Getting the right code

Import the right packages and copy-paste the code of the GAN class

### Step 7 : Import X_face.npy

When using Colab, the file is located in your folder `Drive/My Drive` (with the space). Import the file and define X_train.

### Step 8 : Train the GAN

### Step 9 : Generation !

For this step, you need to generate new faces. What inputs should you give it ? Modify the class and create a "predict" function which takes the required input and outputs a generated face.